In [1]:
import pandas as pd
import csv
import json
import sys
import numpy

import math
from flask import Flask, render_template, request, redirect, Response
import random, json


In [2]:
# gra = {"shubham": [{"id": "heavy feeling", "x": -710, "y": 240, "size": 0.5, "color": "#E57821", "label": "heavy feeling"},{"id": "heavy feeling", "x": -710, "y": 240, "size": 0.5, "color": "#E57821", "label": "heavy feeling"}],
#        "aman":        {"id": "heavy feeling", "x": -710, "y": 240, "size": 0.5, "color": "#E57821", "label": "heavy feeling"}}

# @app.route('/')
# def output():
# 	# serve index template
# 	return render_template('index.html', graph=gra)

In [3]:
# app.run()

# Reading data

In [2]:
original_data = pd.read_csv("Data.csv",encoding = "latin")
data=original_data.set_index("Target")
original_data

,Source,Target,Weight
0,hypertensive disease,vertigo,3363
1,hypertensive disease,syncope,3363
2,hypertensive disease,sweating increased,3363
3,hypertensive disease,sweat,3363
4,hypertensive disease,shortness of breath,3363
...,...,...,...
2124,affect labile,bedridden,45
2125,affect labile,bedridden,45
2126,decubitus ulcer,systolic murmur,42
2127,decubitus ulcer,frail,42


# JSON Creation

In [3]:
df=original_data 
df = df.sample(frac=1).reset_index(drop=True) #shuffling the data
diseases=pd.DataFrame(df['Source'].drop_duplicates(keep='first', inplace=False).reset_index(drop=True)) 
symptoms=pd.DataFrame(df['Target'].drop_duplicates(keep='first', inplace=False).reset_index(drop=True))
symptoms.rename(columns={'Target':'id'},inplace=True)
diseases.rename(columns={'Source':'id'},inplace=True)
symptoms['x']=[*range(-202*10, 203*10, 10)]
symptoms['y']=0
symptoms['size']=0.5
symptoms['color']='#E57821'
symptoms['label']=symptoms['id']
diseases['x']=[*range(-74*20, 75*20,20)]
diseases['y']=1000
diseases['size']=0.5
diseases['color']='#008cc2'
diseases['label']=diseases['id']
symptoms.loc[1::2,"y"] = 80
symptoms.loc[2::3,"y"] = 160
symptoms.loc[3::4,"y"] = 240

diseases.loc[1::2,"y"] = 900
diseases.loc[2::3,"y"] = 800

nodes=symptoms.append(diseases)
#shuffling nodes
nodes=nodes.sample(frac=1).reset_index(drop=True)


df=original_data 
edges=df.rename(columns={'Source':'target','Target':'source'})
edges['type']='curve'
edges['color']='#FF0000'
edges['size']=0.3
edges['id']=[*range(0,2129 ,1)]

In [18]:
temp_frame=pd.DataFrame([*range(0, 809,2)]*math.pi/405)
# temp_frame*=math.pi/405
numpy.cos(temp_frame)*100


TypeError: can't multiply sequence by non-int of type 'float'

# Storing JSON

In [4]:
#dataframe to dictonary
nodes_dict=nodes.to_dict(orient='records')
edges_dict=edges.to_dict(orient='records')

# graphs={ 'nodes' : nodes_dict, 'edges' : edges_dict }
# graph_json= json.dumps(graphs)
# graph_json
# with open("graph.js", "w") as outfile: 
#     outfile.write("graph=") 
#     outfile.write(graph_json)
graphs={}
graphs['nodes']=nodes_dict
graphs['edges']=edges_dict


In [73]:
a=['',"","",'']
a[:] = [x for x in a if x != '']
a

[]

In [21]:
app = Flask(__name__)

@app.route('/')
def output():
	# serve index template
	return render_template('index.html', graphs=graphs)

@app.route('/receiver', methods = ['GET','POST'])
def worker():
	# read json + reply
    data = request.get_json(force=True)
    result = ''
#     print(data)
    print("************************************")

    # for item in data:
    #     result += str(item['make']) + '\n'
    return str(data)

In [22]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Mar/2020 16:27:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2020 16:27:55] "POST /receiver HTTP/1.1" 200 -


************************************


127.0.0.1 - - [25/Mar/2020 16:28:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2020 16:28:15] "POST /receiver HTTP/1.1" 200 -


************************************


# Prediction Module

In [14]:
def Prediction(sy):
    common=data.loc[sy[0]]
    for symptom in sy:
        common=pd.merge(common, data.loc[symptom], how='inner', on=['Source'])
    common=common.iloc[:, :-1*len(sy)]
    common.iloc[:,-1]/=common.iloc[:,-1].sum()
    common.set_axis([*common.columns[:-1], 'Probability'], axis=1, inplace=True)
    common.sort_values(by=['Probability'], ascending=False,inplace=True)
#     print(common.to_dict(orient='records'))
    return(common)


# Taking space sepereted symptoms as input

In [57]:
# hallucinations auditory,feeling hopeless,weepiness,sleeplessness <-----Example input
# sy =input("Enter symptoms").split(',')
shubham=[Prediction]
sy=['pain','headache']
common=shubham[0](sy)
df=original_data
df=df[df['Source'].isin(common['Source'])].reset_index(drop=True)
df=df[df['Target'].isin(sy)].reset_index(drop=True)
df=df.merge(common)
print(df.shape[0])
df
# common

6


,Source,Target,Weight,Probability
0,neoplasm,pain,297,0.460465
1,neoplasm,headache,297,0.460465
2,exanthema,pain,208,0.322481
3,exanthema,headache,208,0.322481
4,sickle cell anemia,pain,140,0.217054
5,sickle cell anemia,headache,140,0.217054
